In [1]:
import keras
from keras import backend as K
from keras.layers import Dense, Input
from keras.models import Model, load_model
from keras.utils import to_categorical
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.datasets import load_boston
import tensorflow as tf
from alibi.explainers import CounterFactualProto

Using TensorFlow backend.


In [2]:
boston = load_boston()
data = boston['data']
target = boston['target']
feature_names = boston['feature_names']

In [3]:
y = np.zeros((target.shape[0],))
y[np.where(target > np.median(target))[0]] = 1

In [4]:
data = np.delete(data, 3, 1)
feature_names = np.delete(feature_names, 3)

In [5]:
mu = data.mean(axis=0)
sigma = data.std(axis=0)
data = (data - mu) / sigma

In [6]:

idx = 475
x_train,y_train = data[:idx,:], y[:idx]
x_test, y_test = data[idx:,:], y[idx:]
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [7]:
np.random.seed(0)
tf.set_random_seed(0)

In [8]:

def nn_model():
    x_in = Input(shape=(12,))
    x = Dense(40, activation='relu')(x_in)
    x = Dense(40, activation='relu')(x)
    x_out = Dense(2, activation='softmax')(x)
    nn = Model(inputs=x_in, outputs=x_out)
    nn.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return nn

In [9]:
nn = nn_model()
nn.summary()
nn.fit(x_train, y_train, batch_size=128, epochs=500, verbose=0)
nn.save('nn_boston.h5')

WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 12)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 40)                520       
_________________________________________________________________
dense_2 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 82        
Total params: 2,242
Trainable params: 2,242
Non-trainable params: 0
_________________________________________________________________


In [11]:
score = nn.evaluate(x_test, y_test, verbose=0)
print('Test accuracy: ', score[1])

Test accuracy:  0.8064516186714172


In [13]:

X = x_test[1].reshape((1,) + x_test[1].shape)
shape = X.shape

In [14]:
# set random seed
np.random.seed(1)
tf.set_random_seed(1)

# define model
nn = load_model('nn_boston.h5')

# get Keras session
sess = K.get_session()

# initialize explainer, fit and generate counterfactual
cf = CounterFactualProto(sess, nn, shape, use_kdtree=True, theta=10., max_iterations=1000,
                         feature_range=(x_train.min(axis=0), x_train.max(axis=0)),
                         c_init=1., c_steps=10)
cf.fit(x_train)
explanation = cf.explain(X)

sess.close()
K.clear_session()

WARNING - From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/alibi/utils/tf.py:26: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.



TypeError: expected sequence object with len >= 0 or a single integer

In [15]:
keras.__version__

'2.3.1'

In [16]:
tf.__version__

'1.15.2'